In [1]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os
from typing import List, Dict , Literal 

load_dotenv()

gemini_api_key = os.getenv("GOOGLE_API_KEY")



class agentGemini:
 

    # Define the function with type hints and docstring
    @staticmethod
    def subtract(a: int, b: int) -> dict:
        """Funzione utile per sottrarre due numeri interi

        Args:
            a: Il primo numero.
            b: Il secondo numero.

        Returns:
            Un dizionario contenente il risultato della sottrazione.
        """
        result = a - b
        return {"result": result}
    
    def _create_function_declaration(self , function : callable) -> Dict:
        """
        Genera automaticamente una dichiarazione (json_schema) di funzione a partire da una funzione Python.
        
        Args:
            function: La funzione Python da cui generare la dichiarazione.
        Returns:
            Una rappresentazione JSON della dichiarazione della funzione.
        """
        return types.FunctionDeclaration.from_callable(
            callable=function,
            client=self.client, 
        ).to_json_dict()
    
    def __init__(self):

        self.client = genai.Client(api_key=gemini_api_key)
        self.model = "gemini-2.5-flash-lite"  

        # Configure the client

        # Create the function declaration
        subtract_declaration = self._create_function_declaration(
            function=self.subtract
        )

        self.tools = types.Tool(function_declarations=[subtract_declaration])

            # Create the configuration with the function
        self.config = types.GenerateContentConfig(
            tools=[self.tools]
        )

        # Deefine user prompt

        contents = types.Content(
            role="user",
            parts=[types.Part(text="quanto fa 7773-32? e quanto fa 12-4?")]
        )
        
        # Make the request
        self.response = self.client.models.generate_content(
            model="gemini-2.5-flash",
            contents=contents,
            config=self.config,
        )

        if self.response.function_calls:
            lista_funzioni_invocate = self.response.function_calls
            for funzione in lista_funzioni_invocate:
                print(funzione.name)

In [2]:
ai = agentGemini()

subtract
subtract


In [3]:
ai.config

GenerateContentConfig(
  tools=[
    Tool(
      function_declarations=[
        FunctionDeclaration(
          description="""Funzione utile per sottrarre due numeri interi

Args:
    a: Il primo numero.
    b: Il secondo numero.

Returns:
    Un dizionario contenente il risultato della sottrazione.""",
          name='subtract',
          parameters=Schema(
            properties={<... 2 items at Max depth ...>},
            required=[<... 2 items at Max depth ...>],
            type=<Type.OBJECT: 'OBJECT'>
          )
        ),
      ]
    ),
  ]
)

In [24]:
ai.response.candidates[0].content.parts

[Part(
   function_call=FunctionCall(
     args={
       'a': 7773,
       'b': 32
     },
     name='subtract'
   ),
   thought_signature=b"\n\xfe\x01\x01\xd1\xed\x8aobI\xc7\xc2U]\xf7\xc4\x19\xb8\x8f\xec(\xbe^\xad\xc36\x0b\xc0\xf8\xcb\xca\xc5T\x10\xb2N\xa8*'\x8b\xac\xc3\xb6\xb4-R:Cq\xd2N_&Cy\xbd\xb5\x95Z\x0cr\xe6mj\x10\xf2+#\xbdgO\xf1\xa3\xf7Lu5\xef\xf6\x16\x16\x9a\x8b=l\x98]!\x90RB\x01\\\xd7\x83\xa4\xca...'
 ),
 Part(
   function_call=FunctionCall(
     args={
       'a': 12,
       'b': 4
     },
     name='subtract'
   )
 )]

In [25]:
diz = ai.response.function_calls[1].args
print(diz)

{'b': 4, 'a': 12}


In [108]:
client =  genai.Client(api_key=gemini_api_key)

def subtract(a: int, b: int) -> dict:
    """Funzione utile per sottrarre due numeri interi

    Args:
        a: Il primo numero.
        b: Il secondo numero.

    Returns:
        Un dizionario contenente il risultato della sottrazione.
    """
    result = a - b
    return {"result": result}

def _create_function_declaration(client , function : callable):
    return types.FunctionDeclaration.from_callable(
        callable=function,
        client=client, 
    )

qqq = _create_function_declaration(client=client , function=subtract)


In [113]:
qqq.to_json_dict()

{'description': 'Funzione utile per sottrarre due numeri interi\n\nArgs:\n    a: Il primo numero.\n    b: Il secondo numero.\n\nReturns:\n    Un dizionario contenente il risultato della sottrazione.',
 'name': 'subtract',
 'parameters': {'properties': {'a': {'type': 'INTEGER'},
   'b': {'type': 'INTEGER'}},
  'required': ['a', 'b'],
  'type': 'OBJECT'}}

In [138]:
ai.response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            ),
            thought_signature=b'\n\x8b\x02\x01\xd1\xed\x8aoX\xccwRKi\x1d\xb6\x05\x1f\xec0\t\xda(\xear\x85\xbe\xd5\xe7f\xdcS\xd3\xa8ZW\xcd\x80&A_\x89\xd9\xe5\xe9y\xd0X|\xae\x0e\xc3Yv\xe0\xbf\x1d\xa7\xf1\xb7=lJ\x84\xb9~\xda1\xc9\x0f\xdaG\xb0\xdd\x07gg\x0e\x0b\xdcs\x01\x8b\xb4*\x07\x0c2\xff\xbd0\x04C\x03\x04\xadc...'
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>,
      index=0
    ),
  ],
  model_version='gemini-2.5-flash',
  response_id='VD4FaaqpF_SAkdUPtYKO0QI',
  sdk_http_response=HttpResponse(
    headers=<dict len=11>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=22,
    prompt_token_count=100,
    prompt_tok

siamo riusciti a far si che venga riconosciuto il function calling
bisogna ora fare

# Check for a function call
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    #  In a real app, you would call your function here:
    #  result = schedule_meeting(**function_call.args)
else:
    print("No function call found in the response.")
    print(response.text)

per runnare la funzione
il risultato va dato come contesto

https://ai.google.dev/gemini-api/docs/function-calling?example=meeting